In [3]:
import json
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import seaborn as sns
sns.set(style='ticks', palette='Set2')
from scipy.interpolate import CubicSpline

In [4]:
with open("./B07cN4_grateffs/B07grating14Apr24.json", 'r') as f:
    data = json.load(f)

In [32]:
data['order'][0]['cff'][1]['energy']

[49.99999999999999,
 50.56122642444571,
 51.12875235088132,
 51.70264848824643,
 52.28298633915507,
 52.86983820880444,
 53.463277213983424,
 54.06337729218251,
 54.67021321080553,
 55.28386057648524,
 55.90439584450313,
 56.5318963283153,
 57.16644020918483,
 57.808106545922634,
 58.45697528473751,
 59.1131272691968,
 59.776644250298716,
 60.44760889665789,
 61.126104804805145,
 61.81221650960304,
 62.50602949477802,
 63.20763020357107,
 63.91710604950779,
 64.63454542728944,
 65.36003772380607,
 66.09367332927346,
 66.83554364849485,
 67.58574111224945,
 68.34435918880828,
 69.11149239557965,
 69.88723631088519,
 70.6716875858682,
 71.46494395653542,
 72.26710425593421,
 73.07826842646625,
 73.89853753233966,
 74.72801377216051,
 75.5668004916661,
 76.41500219660087,
 77.27272456573706,
 78.1400744640412,
 79.01715995598872,
 79.90409031902777,
 80.80097605719449,
 81.70792891488054,
 82.6250618907558,
 83.55248925184686,
 84.49032654777392,
 85.43869062514707,
 86.39769964212458,
 8

In [60]:
data['order'][0]['cff'][4]['s']

[0.04075798894237199,
 0.041099467993735254,
 0.041461999718382415,
 0.04184914054201519,
 0.04225482052380139,
 0.04267088158941572,
 0.04309017014990095,
 0.04349895594171594,
 0.0439096824385138,
 0.04433811314833928,
 0.04479934707196875,
 0.0452931683678204,
 0.045811864985286006,
 0.046361807383620085,
 0.046954025386183734,
 0.04760524888113621,
 0.04826502130965795,
 0.04892877418738396,
 0.049606124852282156,
 0.05031216990583725,
 0.05106779639724588,
 0.05192102039501515,
 0.05273229140147922,
 0.05352265796360087,
 0.05430215119079667,
 0.05507764840228177,
 0.05583384157915742,
 0.05657307809122877,
 0.05729787881808273,
 0.058003673506819085,
 0.05868235304256449,
 0.05930837473738217,
 0.05995996054265899,
 0.06062764153386784,
 0.06130243138205159,
 0.06197810372595122,
 0.06266011484403038,
 0.06336411974889342,
 0.06409877502542051,
 0.06487825700957377,
 0.06570074283056532,
 0.06652818713218626,
 0.06733907041385291,
 0.06813337909096424,
 0.06885652126526753,
 0.06

In [2]:
def get_eff(file, order, E, cff, polarisation='s', extrapolate_energy=True):
    """
    Reads the grating efficiencies from a JSON file.
    
    Parameters
    __________
    
    file: str 
        path to JSON file containing grating efficiencies
    order: int 
        1, 2, 3, 4, or 5
    E: float 
        Energy 50 to 15000 eV
    cff: float 
        cff value from 1.2 to 3 in steps of 0.2
    polarisation: str
        's' for sigma polarisation
        'p' for pi polarisation
        'd' for diagonal polarisation
        'c' for circular polarisation
    extrapolate_energy: bool
        True to return cubic extrapolated efficiency at given E
        False to return an array of energy and efficiency as supplied by file
        
    """
    with open(file, 'r') as f:
        data = json.load(f)
    
    order_index = int(np.abs(order))-1
    cff_index_dict = {f'{x:.1f}': y for x, y in zip(np.arange(1.2,3.2, 0.2), range(0, 11))} 
    cff_index = cff_index_dict[f"{cff}"]
    energy = data['order'][order_index]['cff'][cff_index]['energy']
    eff = data['order'][order_index]['cff'][cff_index][polarisation]
    
    if extrapolate_energy:
        eff_extrapolate = CubicSpline(energy, eff)
        return eff_extrapolate(E)
    else:
        return np.hstack(energy, eff)
        
        

In [81]:
get_eff('./B07cN4_grateffs/B07grating15Mar24.json', 5, 500, 3.)

-5
3


array(0.00559794)

In [42]:
def initial_read(file, polarisation='s'):
    with open(file, 'r') as f:
        data= json.load(f)
    order_len = len(data['order'])
    order_list = [order + 1 for order in range(order_len)]
    cff_dict = {i:[data['order'][i-1]['cff'][j]['cff'] for j in range(len(data['order'][i-1]['cff']))] for i in order_list}
    #energy_dict = {i:data['order'][i-1]['cff']['energy']}
    master_dict = {order+1: 
                   {cff: 
                    np.array([data['order'][order]['cff'][cff_dict[order+1].index(cff)]['energy'], 
                              data['order'][order]['cff'][cff_dict[order+1].index(cff)]['s']]) for cff in cff_dict[order+1]} for order in range(order_len)}
    return order_list, cff_dict, master_dict

In [43]:
a,b,c = initial_read("./B07cN4_grateffs/B07grating14Apr24.json")

In [51]:
data['order'][1]['cff'][0]['s']

[5.801720641414512e-06,
 4.976661386319979e-06,
 4.228256600466755e-06,
 3.558438492244972e-06,
 2.9650803190866465e-06,
 2.442851231195398e-06,
 1.988012082191503e-06,
 1.5906961962599013e-06,
 1.247809117916572e-06,
 9.569501584753957e-07,
 7.15539375990716e-07,
 5.197979133276744e-07,
 3.641469252276489e-07,
 2.441769291647723e-07,
 1.5578329016614573e-07,
 9.47537714932184e-08,
 5.5721436475492036e-08,
 3.441338180361608e-08,
 2.6203179583096032e-08,
 3.4084928487836345e-08,
 6.407753195574934e-08,
 1.096356756491161e-07,
 1.7125282589008295e-07,
 2.496779803913084e-07,
 3.4539951841182626e-07,
 4.581614105703819e-07,
 5.877104007113573e-07,
 7.347662955711122e-07,
 9.003904475638445e-07,
 1.0850726933541916e-06,
 1.287205491615819e-06,
 1.4939952526234956e-06,
 1.7022526595455372e-06,
 1.9230336309102143e-06,
 2.1624567487996947e-06,
 2.4242797744391065e-06,
 2.7072643240789884e-06,
 3.0116398182672714e-06,
 3.350946599858586e-06,
 3.743882691069105e-06,
 4.20389058353752e-06,
 4.

In [53]:
c[2][1.05][1]

array([5.80172064e-06, 4.97666139e-06, 4.22825660e-06, 3.55843849e-06,
       2.96508032e-06, 2.44285123e-06, 1.98801208e-06, 1.59069620e-06,
       1.24780912e-06, 9.56950158e-07, 7.15539376e-07, 5.19797913e-07,
       3.64146925e-07, 2.44176929e-07, 1.55783290e-07, 9.47537715e-08,
       5.57214365e-08, 3.44133818e-08, 2.62031796e-08, 3.40849285e-08,
       6.40775320e-08, 1.09635676e-07, 1.71252826e-07, 2.49677980e-07,
       3.45399518e-07, 4.58161411e-07, 5.87710401e-07, 7.34766296e-07,
       9.00390448e-07, 1.08507269e-06, 1.28720549e-06, 1.49399525e-06,
       1.70225266e-06, 1.92303363e-06, 2.16245675e-06, 2.42427977e-06,
       2.70726432e-06, 3.01163982e-06, 3.35094660e-06, 3.74388269e-06,
       4.20389058e-06, 4.72048866e-06, 5.34040767e-06, 6.01858706e-06,
       6.72033990e-06, 7.44727066e-06, 8.28242351e-06, 9.48711067e-06,
       1.09312395e-05, 1.25064333e-05, 1.41942286e-05, 1.60506279e-05,
       1.81096024e-05, 2.04635029e-05, 2.31796189e-05, 2.63319747e-05,
      